In [9]:
import numpy as np
np.fft.fft([1, 2, 3, 4])

array([10.+0.j, -2.+2.j, -2.+0.j, -2.-2.j])

In [ ]:
from IPython.display import Image
Image("img/picture.png")
"https://github.com/dbindel/cs4220-s20/raw/master/hw/code/proj1/data/blurry.png"